In [3]:
import re
import copy
import numpy as np
import pandas as pd
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [4]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [5]:
import os
file_names = []
# path = os.path.abspath('./My_DBC')
path = os.path.abspath('./opendbc-master/opendbc-master')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
file_names

['D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\acura_ilx_2016_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\acura_ilx_2016_nidec.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\acura_rdx_2018_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\acura_rdx_2020_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\bmw_e9x_e8x.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\cadillac_ct6_chassis.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master\\cadillac_ct6_object.dbc',
 'D:\\

In [6]:
# readFile(file_names[0])
# print(node)
# print(len(node))
# print(messageName)
# print(len(messageName))
# print(SignalsName)
# print(len(SignalsName))
# print(allDatas)
# print(len(allDatas))

In [7]:
# len(allDatas), allDatas

In [14]:
def findOverlap():
    overlapped_message = []
    overlapped_true_message = []
    false_message = []
    flag = False
    for message in allDatas:
        series = np.zeros(64)
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
    #         signedness = signal[4]
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                count = 0
                for i in range(new_start, new_end):
                    if(i>=64):
                        print(message_id,start,length,endianness,new_start, new_end)
                        false_message.append(message_id)
                        flag = True
                        break
                    if series[i] == 1:
                        count = count + 1
                if count > 0 :
                    overlapped_message.append(message_id)
                    flag = True
                if count > 0 & count < length :
                    overlapped_true_message.append(message_id)
                    flag = True
                if flag == True:
                    flag = False
                    break
                series[new_start: new_end] = 1
            if endianness == 1:
                new_end = new_start - length + 1
                count = 0
                for i in range(new_end, new_start + 1):
                    if(i>=64):
                        print(message_id,start,length,endianness,new_start, new_end)
                        false_message.append(message_id)
                        flag = True
                        break
                    if series[i] == 1:
                        count = count + 1
                if count > 0 :
                    overlapped_message.append(message_id)
                    flag = True
                if count > 0 & count < length :
                    overlapped_true_message.append(message_id)
                    flag = True
                if flag == True:
                    flag = False
                    break
                series[new_end: new_start + 1] = 1
    return overlapped_message, overlapped_true_message, false_message

In [15]:
overlap_res = []
false_res = []
overlap_true_res = []
for file_name in file_names:
    cur_name = file_name.split("\\")[-1]
    readFile(file_name)
    cur_overlap, cur_overlap_true, cur_false = findOverlap()
    overlap_res.append({cur_name: cur_overlap})
    false_res.append({cur_name: cur_false})
    overlap_true_res.append({cur_name: cur_overlap_true})

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\acura_ilx_2016_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\acura_ilx_2016_nidec.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\acura_rdx_2018_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\acura_rdx_2020_can_generated.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\bmw_e9x_e8x.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\cadillac_ct6_chassis.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\cadillac_ct6_object.dbc
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master\cad

In [18]:
len(overlap_res), overlap_res

(72,
 [{'acura_ilx_2016_can_generated.dbc': ['829_LKAS_HUD']},
  {'acura_ilx_2016_nidec.dbc': []},
  {'acura_rdx_2018_can_generated.dbc': ['829_LKAS_HUD']},
  {'acura_rdx_2020_can_generated.dbc': ['829_LKAS_HUD',
    '13274_LKAS_HUD_A',
    '13275_LKAS_HUD_B']},
  {'bmw_e9x_e8x.dbc': ['170_AccPedal',
    '191_RequestedWheelTorqueDriveTrainActiveCruiseControl']},
  {'cadillac_ct6_chassis.dbc': []},
  {'cadillac_ct6_object.dbc': []},
  {'cadillac_ct6_powertrain.dbc': []},
  {'chrysler_pacifica_2017_hybrid.dbc': []},
  {'chrysler_pacifica_2017_hybrid_private_fusion.dbc': []},
  {'comma_body.dbc': []},
  {'ESR.dbc': ['1489_SensorValidation2']},
  {'FORD_CADS.dbc': ['1073741824_VECTOR__INDEPENDENT_SIG_MSG']},
  {'ford_cgea1_2_bodycan_2011.dbc': []},
  {'ford_cgea1_2_ptcan_2011.dbc': []},
  {'ford_fusion_2018_adas.dbc': []},
  {'ford_fusion_2018_pt.dbc': []},
  {'ford_lincoln_base_pt.dbc': []},
  {'gm_global_a_chassis.dbc': []},
  {'gm_global_a_high_voltage_management.dbc': ['512_Battery_Mod

In [19]:
len(overlap_true_res), overlap_true_res

(72,
 [{'acura_ilx_2016_can_generated.dbc': ['829_LKAS_HUD']},
  {'acura_ilx_2016_nidec.dbc': []},
  {'acura_rdx_2018_can_generated.dbc': ['829_LKAS_HUD']},
  {'acura_rdx_2020_can_generated.dbc': ['829_LKAS_HUD',
    '13274_LKAS_HUD_A',
    '13275_LKAS_HUD_B']},
  {'bmw_e9x_e8x.dbc': ['170_AccPedal',
    '191_RequestedWheelTorqueDriveTrainActiveCruiseControl']},
  {'cadillac_ct6_chassis.dbc': []},
  {'cadillac_ct6_object.dbc': []},
  {'cadillac_ct6_powertrain.dbc': []},
  {'chrysler_pacifica_2017_hybrid.dbc': []},
  {'chrysler_pacifica_2017_hybrid_private_fusion.dbc': []},
  {'comma_body.dbc': []},
  {'ESR.dbc': ['1489_SensorValidation2']},
  {'FORD_CADS.dbc': ['1073741824_VECTOR__INDEPENDENT_SIG_MSG']},
  {'ford_cgea1_2_bodycan_2011.dbc': []},
  {'ford_cgea1_2_ptcan_2011.dbc': []},
  {'ford_fusion_2018_adas.dbc': []},
  {'ford_fusion_2018_pt.dbc': []},
  {'ford_lincoln_base_pt.dbc': []},
  {'gm_global_a_chassis.dbc': []},
  {'gm_global_a_high_voltage_management.dbc': ['512_Battery_Mod

In [11]:
false_res

[{'acura_ilx_2016_can_generated.dbc': []},
 {'acura_ilx_2016_nidec.dbc': []},
 {'acura_rdx_2018_can_generated.dbc': []},
 {'acura_rdx_2020_can_generated.dbc': []},
 {'bmw_e9x_e8x.dbc': []},
 {'cadillac_ct6_chassis.dbc': []},
 {'cadillac_ct6_object.dbc': []},
 {'cadillac_ct6_powertrain.dbc': []},
 {'chrysler_pacifica_2017_hybrid.dbc': []},
 {'chrysler_pacifica_2017_hybrid_private_fusion.dbc': []},
 {'comma_body.dbc': []},
 {'ESR.dbc': []},
 {'FORD_CADS.dbc': []},
 {'ford_cgea1_2_bodycan_2011.dbc': []},
 {'ford_cgea1_2_ptcan_2011.dbc': []},
 {'ford_fusion_2018_adas.dbc': []},
 {'ford_fusion_2018_pt.dbc': []},
 {'ford_lincoln_base_pt.dbc': []},
 {'gm_global_a_chassis.dbc': []},
 {'gm_global_a_high_voltage_management.dbc': []},
 {'gm_global_a_lowspeed.dbc': []},
 {'gm_global_a_lowspeed_1818125.dbc': []},
 {'gm_global_a_object.dbc': []},
 {'gm_global_a_powertrain_expansion.dbc': []},
 {'gm_global_a_powertrain_generated.dbc': []},
 {'honda_accord_2018_can_generated.dbc': []},
 {'honda_civic_

In [30]:
len({'mazda_2017.dbc': ['1078_HVAC']}.values().values)

1

In [34]:
{'mazda_2017.dbc': ['1078_HVAC']}.values()

dict_values([['1078_HVAC']])

In [ ]:
for i in false_res:
    if(i)